### Mathematical formulations for structure problems.
#### Table of contents
1. [Network generation](#network)
2. [The maximum clique problem](#clique)
    1. [The maximum independent set problem](#independent)
3. [The maximum induced star problem](#star)
4. [The maximum quasi-clique problem](#quasiclique)
5. [The maximum $k$-club problem](#kclub)
6. [Using networkx](#networkx)

### Creating and visualizing the network <a name="network"></a>

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from gurobipy import *
from itertools import combinations

G = nx.karate_club_graph()
pos=nx.spring_layout(G)
nx.draw(G, node_size=30, width=0.5)

#### The maximum clique problem <a name="clique"></a>

$$\max~~\sum_{i\in V} x_i$$
$$\text{s.t}~~x_i+x_j\leq 1, ~\forall \left(i,j\right)\notin E$$
$$~x_i\in\left\{0,1\right\}, ~\forall i\in V.$$

In [ ]:
model=Model("max_clique")
x=model.addVars(G.nodes(), vtype=GRB.BINARY, obj=-1)

for (i,j) in combinations(G.nodes(),2):
    if (i,j) not in G.edges():
        model.addConstr(x[i]+x[j]<=1)
model.optimize()

#### Showing the max clique.

In [ ]:
clique_nodes=[i for i in G.nodes() if x[i].X==1]
clique_edges=[(i,j) for (i,j) in G.edges() if i in clique_nodes and j in clique_nodes]

nx.draw(G, pos, node_size=30, width=0.5, alpha=0.3)
nx.draw_networkx_nodes(G, pos, nodelist=clique_nodes, node_color='r', node_size=40)
nx.draw_networkx_edges(G, pos, edgelist=clique_edges, edge_color='r', width=0.75)

#### The maximum independent set problem <a name="independent"></a>

$$\max~~\sum_{i\in V} x_i$$
$$\text{s.t}~~x_i+x_j\leq 1, ~\forall \left(i,j\right)\in E$$
$$~x_i\in\left\{0,1\right\}, ~\forall i\in V.$$

In [ ]:
model=Model("max_independent_set")
x=model.addVars(G.nodes(), vtype=GRB.BINARY, obj=-1)

for (i,j) in G.edges():
    model.addConstr(x[i]+x[j]<=1)
model.optimize()

IS_nodes=[i for i in G.nodes() if x[i].X==1]

nx.draw(G, pos, node_size=30, width=0.5, alpha=0.3)
nx.draw_networkx_nodes(G, pos, nodelist=IS_nodes, node_color='r', node_size=40)

#### The maximum induced star problem <a name="star"></a>

In [ ]:
model=Model("max_star")
y=model.addVars(G.nodes(), vtype=GRB.BINARY, obj=-1)
x=model.addVars(G.nodes(), vtype=GRB.BINARY, obj=-1)

for (i,j) in G.edges():
    model.addConstr(y[i]+y[j]<=1)

for i in G.nodes():
    model.addConstr(y[i]<=quicksum(x[j] for j in G.neighbors(i)))

model.addConstr(quicksum(x[i] for i in G.nodes())==1)
    
model.optimize()

In [ ]:
star_nodes=[i for i in G.nodes() if y[i].X==1]
center=[i for i in G.nodes() if x[i].X==1]
star_edges=[(i,j) for (i,j) in G.edges() if i in star_nodes and j in center]

nx.draw(G, pos, node_size=30, width=0.5, alpha=0.3)
nx.draw_networkx_nodes(G, pos, nodelist=star_nodes, node_color='r', node_size=40)
nx.draw_networkx_nodes(G, pos, nodelist=center, node_color='b', node_size=40)
nx.draw_networkx_edges(G, pos, edgelist=star_edges, edge_color='r', width=0.75)

#### The maximum quasi-clique problem <a name="quasiclique"></a>

In [ ]:
gamma=0.7
model=Model("max_quasi_clique")
w={}
for (i,j) in combinations(G.nodes(),2):
    w[i,j]=model.addVar(vtype=GRB.BINARY)
x=model.addVars(G.nodes(), vtype=GRB.BINARY, obj=-1)

for (i,j) in combinations(G.nodes(),2):
    model.addConstr(w[i,j]<=x[i])
    model.addConstr(w[i,j]<=x[j])
    model.addConstr(w[i,j]>=x[i]+x[j]-1)

model.addConstr(quicksum(w[i,j] for (i,j) in G.edges())>=gamma*quicksum(w[i,j] for (i,j) in combinations(G.nodes(),2)))
    
model.optimize()

In [ ]:
quasi_clique_nodes=[i for i in G.nodes() if x[i].X==1]
quasi_clique_edges=[(i,j) for (i,j) in G.edges() if i in quasi_clique_nodes and j in quasi_clique_nodes]

nx.draw(G, pos, node_size=30, width=0.5, alpha=0.3)
nx.draw_networkx_nodes(G, pos, nodelist=quasi_clique_nodes, node_color='r', node_size=40)
nx.draw_networkx_edges(G, pos, edgelist=quasi_clique_edges, edge_color='r', width=0.75)

#### The maximum $\boldsymbol{k}$-club problem <a name="kclub"></a>

In [ ]:
paths={}
k=2
model=Model("max_k_club")
for (i,j) in combinations(G.nodes(),2):
    paths[i,j] = list(nx.all_simple_paths(G, source=i, target=j, cutoff=k))
    
x=model.addVars(G.nodes(), vtype=GRB.BINARY, obj=-1, name="x"+str(i))
y={}
for (i,j) in combinations(G.nodes(),2):
    for p in paths[i,j]:
        y[tuple(p)]=model.addVar(vtype=GRB.BINARY, obj=0, name="y"+str(p))

for (i,j) in combinations(G.nodes(),2):
    model.addConstr(x[i]+x[j]<=1+quicksum(y[tuple(p)] for p in paths[i,j]))
for (i,j) in combinations(G.nodes(),2):
    for p in paths[i,j]:
        for ell in p:
            model.addConstr(x[ell]>=y[tuple(p)])
model.write("test.lp")
model.optimize()

In [ ]:
k_club_nodes=[i for i in G.nodes() if x[i].X==1]
k_club_edges=[(i,j) for (i,j) in G.edges() if i in k_club_nodes and j in k_club_nodes]

nx.draw(G, pos, node_size=30, width=0.5, alpha=0.3)
nx.draw_networkx_nodes(G, pos, nodelist=k_club_nodes, node_color='r', node_size=40)
nx.draw_networkx_edges(G, pos, edgelist=k_club_edges, edge_color='r', width=0.75)

#### Cliques and independent sets can also be found through ``networkx`` <a name="networkx"></a>

In [ ]:
from networkx.algorithms import approximation
IS_nodes=approximation.maximum_independent_set(G)
print(IS_nodes)
clique_nodes=approximation.max_clique(G)
print(clique_nodes)

In [ ]:
clique_edges=[(i,j) for (i,j) in G.edges() if i in clique_nodes and j in clique_nodes]
nx.draw(G, pos, node_size=30, width=0.5, alpha=0.3)
nx.draw_networkx_nodes(G, pos, nodelist=clique_nodes, node_color='r', node_size=40)
nx.draw_networkx_edges(G, pos, edgelist=clique_edges, edge_color='r', width=0.75)

In [ ]:
nx.draw(G, pos, node_size=30, width=0.5, alpha=0.3)
nx.draw_networkx_nodes(G, pos, nodelist=IS_nodes, node_color='r', node_size=40)